# Filter movies by genre

In [1]:
titles = open('../Datasets/cornell movie-dialogs corpus/movie_titles_metadata.txt', 'rb')
titles_str = str(titles.read())[2:]
titles.close()

In [23]:
split = titles_str.split("+++$+++")
movies = 617
genres_not_allowed = []
genres_allowed = ["romance"]

In [24]:
movies_to_scrape = []
for movie in range(movies):
    
    save = False
    
    # Get movie id
    _id = split[movie*5].split("nm")[-1].replace(" ", "")
    if "m" not in _id:
        _id = "m"+_id
    
    # Get genre
    genre = split[movie*5].split("nm")[0]
    for a in genres_allowed:
        if a in genre:
            save = True
            
#     for na in genres_not_allowed:
#         if na in genre:
#             save = False
    
    # Save
    if save:
        movies_to_scrape.append(_id)

print("{} movies to use.".format(len(movies_to_scrape)))

132 movies to use.


# Get raw dialouge

In [25]:
# extract raw dialouge
lines = open('../Datasets/cornell movie-dialogs corpus/movie_lines.txt', 'rb')
lines_str = str(lines.read())[2:]
lines.close()

In [26]:
import itertools
split = [char.split("\\n") for char in lines_str.split("+++$+++")]
merged = list(itertools.chain(*split))

In [27]:
dialouge = {} # movie_id : {c1: <text>, c2: <text>}
for movie in movies_to_scrape:
    dialouge[movie]={}

In [28]:
%%time
c = 0
while c+4 < len(merged)-1:
    movie = merged[c+2].replace(" ", "")
    char = merged[c+1].replace(" ", "")
    text = merged[c+4]
    line = merged[c].replace(" ", "")
    
    if movie in movies_to_scrape:
        if char not in dialouge[movie]:
            dialouge[movie][char] = {}
        dialouge[movie][char].update({line : text})
        
    c+=5

CPU times: user 867 ms, sys: 0 ns, total: 867 ms
Wall time: 863 ms


In [29]:
dialouge.keys()

dict_keys(['m1', 'm6', 'm13', 'm14', 'm18', 'm25', 'm29', 'm32', 'm36', 'm43', 'm45', 'm56', 'm57', 'm60', 'm78', 'm79', 'm85', 'm89', 'm91', 'm95', 'm103', 'm104', 'm117', 'm119', 'm120', 'm122', 'm131', 'm133', 'm138', 'm141', 'm150', 'm153', 'm155', 'm162', 'm164', 'm165', 'm168', 'm169', 'm172', 'm179', 'm186', 'm188', 'm189', 'm200', 'm208', 'm209', 'm218', 'm221', 'm230', 'm241', 'm242', 'm244', 'm247', 'm250', 'm252', 'm253', 'm255', 'm256', 'm258', 'm268', 'm279', 'm280', 'm286', 'm289', 'm294', 'm298', 'm304', 'm313', 'm317', 'm335', 'm336', 'm337', 'm343', 'm345', 'm353', 'm366', 'm376', 'm377', 'm381', 'm396', 'm403', 'm407', 'm409', 'm415', 'm416', 'm424', 'm426', 'm427', 'm428', 'm436', 'm445', 'm447', 'm451', 'm452', 'm453', 'm455', 'm456', 'm459', 'm463', 'm465', 'm466', 'm468', 'm469', 'm470', 'm480', 'm481', 'm487', 'm492', 'm494', 'm499', 'm500', 'm504', 'm512', 'm514', 'm521', 'm524', 'm543', 'm560', 'm562', 'm565', 'm566', 'm568', 'm570', 'm574', 'm575', 'm576', 'm5

# Construct structured dialogue

In [30]:
# extract structure
structure = open('../Datasets/cornell movie-dialogs corpus/movie_conversations.txt', 'rb')
structure_str = str(structure.read())[2:]
structure.close()

In [31]:
import itertools
structure_str = [char.split("\\n") for char in structure_str.split("+++$+++")]
structure_str = list(itertools.chain(*structure_str))

In [32]:
structure_str[:10]

['u0 ',
 ' u2 ',
 ' m0 ',
 " ['L194', 'L195', 'L196', 'L197']",
 'u0 ',
 ' u2 ',
 ' m0 ',
 " ['L198', 'L199']",
 'u0 ',
 ' u2 ']

In [33]:
%%time
from itertools import cycle
structured_dialogue = []

c = 0
while c+3 < len(structure_str)-1:
    movie = structure_str[c+2].replace(" ", "")
    char1 = structure_str[c].replace(" ", "")
    char2 = structure_str[c+1].replace(" ", "")
    chars = cycle([char1, char2])
    lines = structure_str[c+3].replace(" ", "").replace("[", "").replace("]", "").replace("'","").split(",")
    if len(lines) % 2 != 0:
        lines = lines[:-1]
    
    if movie in movies_to_scrape:
    
        for line in lines:
            try:
                char = next(chars)
                structured_dialogue.append(dialouge[movie][char][line])
            except:
                char = next(chars)
                structured_dialogue.append(dialouge[movie][char][line])

    c+=4

CPU times: user 423 ms, sys: 0 ns, total: 423 ms
Wall time: 419 ms


# Filter keywords

In [34]:
keywords_i = ["I", "my", "me", "My"]
keywords_o = ["you",  "You", "your", "Your"]
keys = cycle([keywords_i, keywords_o])

In [35]:
len(structured_dialogue)

59352

In [36]:
filtered_dialogue = []
for i in range(0, len(structured_dialogue)-1, 2):
    input_sentence = None
    output_sentence = None
    
    for key in keywords_i:
        if key in structured_dialogue[i]:
            input_sentence = structured_dialogue[i]
            output_sentence = structured_dialogue[i+1]
            break
        if key in structured_dialogue[i+1]:
            input_sentence = structured_dialogue[i+1]
            output_sentence = structured_dialogue[i]
            break
            
    if not input_sentence:
        for key in keywords_o:
            if key in structured_dialogue[i+1]:
                output_sentence = structured_dialogue[i+1]
                input_sentence = structured_dialogue[i]
                break
        for key in keywords_o:
            if key in structured_dialogue[i]:
                output_sentence = structured_dialogue[i]
                input_sentence = structured_dialogue[i+1]
                break
        
    if output_sentence and input_sentence:
        filtered_dialogue.append(input_sentence[1:])
        filtered_dialogue.append(output_sentence[1:])

In [37]:
len(filtered_dialogue)

51942

In [38]:
import pickle
f = open("./romance_dialogue.pickle", "wb")
pickle.dump(filtered_dialogue, f)

In [39]:
filtered_dialogue

['I never seen heat like this!  Not even in Las Minas!',
 "The water\\'s going putrid in the barrels.",
 "Ah, leave him alone.  He\\'s doing no harm.",
 'What are you listening to, chicken ass?',
 'Yes, your Eminence.  The voyage should not take more than six or seven weeks.',
 'You say Asia can be found by sailing west?',
 'I am, Your Eminence',
 'Unfortunately, Don Colon, that is precisely where our opinions differ...  Are you familiar with the work of Aristotle?  Erathostene?  Ptolemeus?',
 'Senor Colon, an experienced captain such as yourself will understand our concern with the crew.  I am not willing to have on my conscience the loss of men who would have relied upon our judgment.',
 'Excellency, you are right.',
 'Your Eminence, there is only one way to settle the matter.  And that is to make the journey.  I am ready to risk my life to prove it possible.',
 'Your life, and that of others!',
 'Is that all that interests you? Gold?',
 'Trade, Your Excellency.  According to Marco P